In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud


In [38]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [39]:
df = pd.read_csv('DataSet_Emails.csv')


In [40]:
print(df.head())


   Unnamed: 0  message_id                                               text  \
0           0       33214  any software just for 15 $ - 99 $ understandin...   
1           1       11929  perspective on ferc regulatory action client c...   
2           2       19784  wanted to try ci 4 lis but thought it was way ...   
3           3        2209  enron / hpl actuals for december 11 , 2000 tec...   
4           4       15880  looking for cheap high - quality software ? ro...   

   label label_text                                            subject  \
0      1       spam                  any software just for 15 $ - 99 $   
1      0        ham  perspective on ferc regulatory action client c...   
2      1       spam  wanted to try ci 4 lis but thought it was way ...   
3      0        ham         enron / hpl actuals for december 11 , 2000   
4      1       spam  looking for cheap high - quality software ? ro...   

                                             message        date  
0  unde

In [41]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31716 entries, 0 to 31715
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  31716 non-null  int64 
 1   message_id  31716 non-null  int64 
 2   text        31665 non-null  object
 3   label       31716 non-null  int64 
 4   label_text  31716 non-null  object
 5   subject     31442 non-null  object
 6   message     31371 non-null  object
 7   date        31716 non-null  object
dtypes: int64(3), object(5)
memory usage: 1.9+ MB
None


In [42]:
print(df.isnull().sum())


Unnamed: 0      0
message_id      0
text           51
label           0
label_text      0
subject       274
message       345
date            0
dtype: int64


In [43]:
df.duplicated().sum()

np.int64(0)

In [44]:
print(df['label'].value_counts())

label
1    16163
0    15553
Name: count, dtype: int64


In [45]:

# # Visualiser la distribution
# df['label'].value_counts().plot(kind='bar', color=['green', 'red'])
# plt.title('Distribution des emails (Ham vs Spam)')
# plt.xlabel('Type')
# plt.ylabel('Nombre')
# plt.xticks(rotation=0)
# plt.savefig('DistributionDesEmails(HamVSSpam)')
# plt.close()

In [46]:
spam_text = " ".join(df[df['label_text'] == 'spam']['text'].astype(str))
ham_text = " ".join(df[df['label_text'] == 'ham']['text'].astype(str))



In [47]:
# wc_spam = WordCloud(width=800, height=400, background_color='white').generate(spam_text)
# plt.figure(figsize=(10,5))
# plt.imshow(wc_spam)
# plt.axis('off')
# plt.title("Mots fréquents dans les spams")
# plt.savefig('MotsFréquentsDansLesSpams')
# plt.close()

# wc_ham = WordCloud(width=800, height=400, background_color='white').generate(ham_text)
# plt.figure(figsize=(10,5))
# plt.imshow(wc_ham)
# plt.axis('off')
# plt.title("Mots fréquents dans les hams")
# plt.savefig('MotsFréquentsDansLesHams')
# plt.close()


In [48]:
df['text'].str.lower()

0        any software just for 15 $ - 99 $ understandin...
1        perspective on ferc regulatory action client c...
2        wanted to try ci 4 lis but thought it was way ...
3        enron / hpl actuals for december 11 , 2000 tec...
4        looking for cheap high - quality software ? ro...
                               ...                        
31711    credit netco start up plan louise ,\nattached ...
31712    everything you are looking for hello , visit o...
31713    start date : 2 / 5 / 02 ; hourahead hour : 18 ...
31714    corhshucker daren - - - -\nthe invoices are be...
31715    re : prosym license hi karolina ,\nthe last wo...
Name: text, Length: 31716, dtype: object

In [49]:
df = df.drop_duplicates()


In [50]:
df = df.dropna(subset=['text'])

In [51]:
df = df[df['text'].str.strip() != '']

In [52]:
# nltk.download('punkt')



In [53]:
# nltk.download('stopwords')


In [54]:
print(nltk.data.path)



['C:\\Users\\abirm/nltk_data', 'c:\\Users\\abirm\\Projects\\SpamShieldAI\\venv\\nltk_data', 'c:\\Users\\abirm\\Projects\\SpamShieldAI\\venv\\share\\nltk_data', 'c:\\Users\\abirm\\Projects\\SpamShieldAI\\venv\\lib\\nltk_data', 'C:\\Users\\abirm\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data', 'c:\\Users\\abirm\\Projects\\SpamShieldAI\\nltk_data']


In [55]:
import os 
# Créer un dossier nltk_data dans votre venv
nltk_data_dir = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)

# Ajouter ce chemin
nltk.data.path.append(nltk_data_dir)

# Télécharger punkt dans ce dossier
nltk.download('punkt', download_dir=nltk_data_dir)
nltk.download('punkt_tab', download_dir=nltk_data_dir)


[nltk_data] Downloading package punkt to
[nltk_data]     c:\Users\abirm\Projects\SpamShieldAI\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     c:\Users\abirm\Projects\SpamShieldAI\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [56]:
nltk.download('stopwords', download_dir=nltk_data_dir)

[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\abirm\Projects\SpamShieldAI\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
from nltk.tokenize import word_tokenize

df['text'] = df['text'].astype(str).apply(word_tokenize)


In [58]:
df['text'].astype(str).head()

0    ['any', 'software', 'just', 'for', '15', '$', ...
1    ['perspective', 'on', 'ferc', 'regulatory', 'a...
2    ['wanted', 'to', 'try', 'ci', '4', 'lis', 'but...
3    ['enron', '/', 'hpl', 'actuals', 'for', 'decem...
4    ['looking', 'for', 'cheap', 'high', '-', 'qual...
Name: text, dtype: object

In [59]:
stop_words = set(stopwords.words('english'))


In [60]:
df['text'] = df['text'].apply(lambda tokens: [t for t in tokens if t.lower() not in stop_words])


In [61]:
import re

df['text'] = df['text'].apply(
    lambda tokens: [t for t in tokens if re.match(r'^[A-Za-z0-9]+$', t)]
)

In [62]:
stemmer = PorterStemmer()


In [63]:
df['text'] = df['text'].apply(
    lambda tokens: [stemmer.stem(t) for t in tokens]
)

In [64]:
df['text'].astype(str).head()

0    ['softwar', '15', '99', 'understand', 'oem', '...
1    ['perspect', 'ferc', 'regulatori', 'action', '...
2    ['want', 'tri', 'ci', '4', 'li', 'thought', 'w...
3    ['enron', 'hpl', 'actual', 'decemb', '11', '20...
4    ['look', 'cheap', 'high', 'qualiti', 'softwar'...
Name: text, dtype: object

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    tokenizer=lambda x: x,  
    preprocessor=lambda x: x, 
    token_pattern=None        
)

X = tfidf.fit_transform(df['text'])

In [66]:
# print(X)

In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, df['label'], test_size=0.2, random_state=42
)


In [68]:
from sklearn.metrics import accuracy_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [69]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=300),
    "SVM (Linear)": SVC(kernel="linear"),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(n_estimators=200),
    "Gradient Boosting": GradientBoostingClassifier(),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

In [70]:
for name, model in models.items():
    print(f'=================== \n model {name}')
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, preds))
    print(classification_report(y_test, preds))

 model Logistic Regression
Accuracy: 0.9851571135322912
              precision    recall  f1-score   support

           0       1.00      0.97      0.98      3120
           1       0.97      1.00      0.99      3213

    accuracy                           0.99      6333
   macro avg       0.99      0.98      0.99      6333
weighted avg       0.99      0.99      0.99      6333

 model SVM (Linear)
Accuracy: 0.9902100110532134
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      3120
           1       0.98      1.00      0.99      3213

    accuracy                           0.99      6333
   macro avg       0.99      0.99      0.99      6333
weighted avg       0.99      0.99      0.99      6333

 model Naive Bayes
Accuracy: 0.9846834043897047
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3120
           1       0.98      0.99      0.98      3213

    accuracy                 

In [71]:
svm_model = models["SVM (Linear)"]


In [72]:
import joblib

joblib.dump(svm_model, "svm_model.pkl")


['svm_model.pkl']